In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations

# Load dataset
file_path = "Bakery.csv"
df = pd.read_csv(file_path)

# Convert data into a list of transactions, ensuring all items are strings
transactions = df.dropna().applymap(str).values.tolist()

# Convert transactions into a dictionary of TID sets
item_tid_sets = {}
for tid, transaction in enumerate(transactions):
    for item in set(transaction):  # Use set to avoid duplicate items in a transaction
        if item not in item_tid_sets:
            item_tid_sets[item] = set()
        item_tid_sets[item].add(tid)

# Function to generate frequent itemsets using ECLAT
def eclat(prefix, items, min_support, results):
    while items:
        item, tid_set = items.pop()
        support = len(tid_set) / len(transactions)
        if support >= min_support:
            new_prefix = prefix + [item]
            results.append((new_prefix, support))
            
            # Create new items dictionary for recursive calls
            new_items = []
            for other_item, other_tid_set in items:
                new_tid_set = tid_set & other_tid_set  # Intersection of TID sets
                if len(new_tid_set) / len(transactions) >= min_support:
                    new_items.append((other_item, new_tid_set))
                    
            eclat(new_prefix, new_items, min_support, results)

# Run ECLAT with min_support = 0.1
min_support = 0.1
results = []
items = list(item_tid_sets.items())
eclat([], items, min_support, results)

# Convert results to a DataFrame
frequent_itemsets_eclat = pd.DataFrame(results, columns=["Itemset", "Support"])

# Sort frequent itemsets by support (descending order)
frequent_itemsets_eclat = frequent_itemsets_eclat.sort_values(by='Support', ascending=False)

# Display sorted frequent itemsets
print("Frequent Itemsets (ECLAT):")
print(frequent_itemsets_eclat)

Frequent Itemsets (ECLAT):
                 Itemset   Support
0              [Weekday]  0.624518
5            [Afternoon]  0.564149
10             [Morning]  0.409811
13             [Weekend]  0.375482
1   [Weekday, Afternoon]  0.354659
8               [Coffee]  0.266787
3     [Weekday, Morning]  0.252304
7   [Afternoon, Weekend]  0.209489
2      [Weekday, Coffee]  0.172770
12               [Bread]  0.162140
11    [Morning, Weekend]  0.157507
6    [Afternoon, Coffee]  0.137660
9      [Coffee, Morning]  0.124884
4       [Weekday, Bread]  0.102014


C:\Users\Jain\AppData\Local\Temp\ipykernel_4592\3566200062.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  transactions = df.dropna().applymap(str).values.tolist()
